In [ ]:
import random
from datetime import datetime, timedelta
import sqlite3

In [ ]:
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Product (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    category TEXT,
    discontinued BOOLEAN
)''')


In [ ]:

cursor.execute('''
CREATE TABLE IF NOT EXISTS Variant (
    variant_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER,
    name TEXT,
    price REAL,
    discontinued BOOLEAN,
    FOREIGN KEY (product_id) REFERENCES Product (product_id)
)''')


In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customer (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT,
    phone TEXT,
    address TEXT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
)''')


In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS "OrderItem" (
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER,
    variant_id INTEGER,
    quantity INTEGER,
    price_at_purchase REAL,
    FOREIGN KEY (order_id) REFERENCES "Order" (order_id),
    FOREIGN KEY (variant_id) REFERENCES Variant (variant_id)
)''')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS "Order" (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    order_date TIMESTAMP,
    total_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES Customer (customer_id)
)''')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS CustomerAddress (
    address_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    address TEXT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY (customer_id) REFERENCES Customer (customer_id)
)''')

In [13]:
categories = ['clothing', 'groceries', 'electronics']
products = [
    {'name': 'T-Shirt', 'category': 'clothing'},
    {'name': 'Jeans', 'category': 'clothing'},
    {'name': 'Laptop', 'category': 'electronics'},
    {'name': 'Headphones', 'category': 'electronics'},
    {'name': 'Bread', 'category': 'groceries'},
    {'name': 'Milk', 'category': 'groceries'},
    {'name': 'Smartphone', 'category': 'electronics'},
    {'name': 'Jacket', 'category': 'clothing'},
    {'name': 'Cereal', 'category': 'groceries'},
    {'name': 'TV', 'category': 'electronics'}
]



In [14]:
customers = [
    {'name': 'Alice', 'email': 'alice@example.com', 'phone': '1234567890', 'address': '123 Main St'},
    {'name': 'Bob', 'email': 'bob@example.com', 'phone': '2345678901', 'address': '456 Elm St'},
    {'name': 'Charlie', 'email': 'charlie@example.com', 'phone': '3456789012', 'address': '789 Oak St'},
    {'name': 'David', 'email': 'david@example.com', 'phone': '4567890123', 'address': '101 Pine St'},
    {'name': 'Eve', 'email': 'eve@example.com', 'phone': '5678901234', 'address': '202 Maple St'},
    {'name': 'Frank', 'email': 'frank@example.com', 'phone': '6789012345', 'address': '303 Birch St'},
    {'name': 'Grace', 'email': 'grace@example.com', 'phone': '7890123456', 'address': '404 Cedar St'},
    {'name': 'Heidi', 'email': 'heidi@example.com', 'phone': '8901234567', 'address': '505 Spruce St'},
    {'name': 'Ivan', 'email': 'ivan@example.com', 'phone': '9012345678', 'address': '606 Fir St'},
    {'name': 'Judy', 'email': 'judy@example.com', 'phone': '1234567899', 'address': '707 Willow St'}
]


In [15]:
for product in products:
    cursor.execute('INSERT INTO Product (name, category, discontinued) VALUES (?, ?, ?)',
                   (product['name'], product['category'], False))

In [16]:
variants = [
    {'product_id': 1, 'name': 'Red', 'price': 10.0, 'discontinued': False},
    {'product_id': 1, 'name': 'Green', 'price': 12.0, 'discontinued': False},
    {'product_id': 2, 'name': 'Blue', 'price': 15.0, 'discontinued': False},
    {'product_id': 3, 'name': '16GB RAM', 'price': 1000.0, 'discontinued': False},
    {'product_id': 3, 'name': '8GB RAM', 'price': 900.0, 'discontinued': False},
    {'product_id': 4, 'name': 'Wireless', 'price': 50.0, 'discontinued': False},
    {'product_id': 5, 'name': 'Whole Grain', 'price': 3.0, 'discontinued': False},
    {'product_id': 6, 'name': 'Organic', 'price': 4.0, 'discontinued': False},
    {'product_id': 7, 'name': '64GB', 'price': 700.0, 'discontinued': False},
    {'product_id': 8, 'name': 'Large', 'price': 60.0, 'discontinued': False},
    {'product_id': 9, 'name': 'Honey Nut', 'price': 5.0, 'discontinued': False},
    {'product_id': 10, 'name': '4K', 'price': 500.0, 'discontinued': False}
]

In [17]:
for variant in variants:
    cursor.execute('INSERT INTO Variant (product_id, name, price, discontinued) VALUES (?, ?, ?, ?)',
                   (variant['product_id'], variant['name'], variant['price'], variant['discontinued']))

In [18]:
for customer in customers:
    created_at = datetime.now() - timedelta(days=random.randint(30, 365*2))
    updated_at = datetime.now()
    cursor.execute('INSERT INTO Customer (name, email, phone, address, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?)',
                   (customer['name'], customer['email'], customer['phone'], customer['address'], created_at, updated_at))

In [19]:
for _ in range(100):  
    customer_id = random.randint(1, len(customers))
    order_date = datetime.now() - timedelta(days=random.randint(1, 365*2))
    total_amount = 0
    cursor.execute('INSERT INTO "Order" (customer_id, order_date, total_amount) VALUES (?, ?, ?)',
                   (customer_id, order_date, total_amount))
    order_id = cursor.lastrowid
    
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        variant_id = random.randint(1, len(variants))
        quantity = random.randint(1, 3)
        price_at_purchase = cursor.execute('SELECT price FROM Variant WHERE variant_id = ?', (variant_id,)).fetchone()[0]
        total_amount += price_at_purchase * quantity
        cursor.execute('INSERT INTO OrderItem (order_id, variant_id, quantity, price_at_purchase) VALUES (?, ?, ?, ?)',
                       (order_id, variant_id, quantity, price_at_purchase))
    
    cursor.execute('UPDATE "Order" SET total_amount = ? WHERE order_id = ?', (total_amount, order_id))

In [20]:
for customer_id in range(1, len(customers) + 1):
    address = cursor.execute('SELECT address FROM Customer WHERE customer_id = ?', (customer_id,)).fetchone()[0]
    created_at = datetime.now() - timedelta(days=random.randint(30, 365*2))
    updated_at = datetime.now()
    cursor.execute('INSERT INTO CustomerAddress (customer_id, address, created_at, updated_at) VALUES (?, ?, ?, ?)',
                   (customer_id, address, created_at, updated_at))